#### (1) cv2.dft()

> cv2.dft()는 실수부와 허수부로 구성된 두 개의 채널 데이터(실수)를 출력


In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('../data/lena.png',0).astype(np.float32)/255

dft = cv2.dft(img, flags=cv2.DFT_COMPLEX_OUTPUT)

print(dft.shape, dft.dtype)
#2채널인 이유는 실수부 허수부
#uint8로는 dft 못함



(512, 512, 2) float32


In [4]:
dft_shift = np.fft.fftshift(dft)
magnitude_spectrum = 20*np.log10(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))

cv2.imshow('magnitude_spectrum', magnitude_spectrum/np.max(magnitude_spectrum))
cv2.waitKey()
cv2.destroyAllWindows()


#### (2) np.fft.fft2()
> np.fft.fft2()는 복소수 1채널 데이터를 출력

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('../data/lena.png',0).astype(np.float32)/255

fft = np.fft.fft2(img)

print(fft.shape, 'of', fft.dtype)

# (0,0) frequency: centered
fft_shift = np.fft.fftshift(fft, axes=[0, 1])

magnitude_spectrum = 20*np.log10(np.abs(fft_shift))

cv2.imshow('magnitude_spectrum', magnitude_spectrum/np.max(magnitude_spectrum))
cv2.waitKey()
cv2.destroyAllWindows()


(512, 512) of complex128


#### (3) Frequency-Domain Lowpass Filtering (Zonsl Filter)

In [54]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('../data/lena.png',0).astype(np.float32)/255

rows, cols = img.shape[:2]

d = cols//5

mask = np.zeros((rows,cols),np.uint8)
mask = cv2.circle(mask,(cols//2,rows//2), d, 1, -1)

fft = np.fft.fft2(img)

# (0,0) frequency: centered
fft_shift = np.fft.fftshift(fft, axes=[0, 1])

# apply mask and inverse DFT
fshift = fft_shift*mask

magnitude_spectrum = 20*np.log10(np.abs(fft_shift))
magnitude_spectrum /= np.max(magnitude_spectrum)
filtered_spectrum = 20*np.log10(np.abs(fshift))
filtered_spectrum /= np.max(filtered_spectrum)

fft_ishift = np.fft.ifftshift(fft_shift)
f_ishift = np.fft.ifftshift(fshift)

img0_back = np.abs(np.fft.ifft2(fft_ishift)).astype(np.float32)
img1_back = np.abs(np.fft.ifft2(f_ishift)).astype(np.float32)

psnr0 = cv2.PSNR(img, img0_back, 1)
psnr1 = cv2.PSNR(img, img1_back, 1)

cv2.imshow('{:0.2f} dB vs {:0.2f} dB'.format(psnr0,psnr1),\
    np.vstack([np.hstack([magnitude_spectrum,filtered_spectrum]),np.hstack([img0_back,img1_back])]))
cv2.waitKey()
cv2.destroyAllWindows()

C:\Users\dwkang\AppData\Local\Temp\ipykernel_13192\3853838238.py:24: RuntimeWarning: divide by zero encountered in log10
  filtered_spectrum = 20*np.log10(np.abs(fshift))


#### (4) Other Masks

In [56]:
sz2 = fft.shape[0]
mask2 = np.ones(fft.shape[:2], np.float32)
mask2 = cv2.circle(mask2,(0,0), int(sz2*1.414/2.5), 0, -1)
mask2 = cv2.circle(mask2,(0,sz2-1), int(sz2*1.414/2.5), 0, -1)
mask2 = cv2.circle(mask2,(sz2-1,0), int(sz2*1.414/2.5), 0, -1)
mask2 = cv2.circle(mask2,(sz2-1,sz2-1), int(sz2*1.414/2.5), 0, -1)

cv2.imshow('mask', mask2)
cv2.waitKey()
cv2.destroyAllWindows()

fft_shift_lpf2 = fft_shift * mask2
fft_lpf2 = np.fft.ifftshift(fft_shift_lpf2, axes=[0, 1])
filtered_lpf2 = np.fft.ifft2(fft_lpf2)
filtered_lpf2 = np.abs(filtered_lpf2)

cv2.imshow('filtered vs filtered2', np.hstack([img1_back,filtered_lpf2]) )
cv2.waitKey()
cv2.destroyAllWindows()

#### (5) Freq-Domain Filtering of Color Image

In [36]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#컬러는 rgb분해후 필터링 따로

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised =(image + noise).clip(0,1)
filtered = np.zeros_like(noised)

sz1 = fft.shape[0]//8
mask1 = np.zeros(fft.shape[:2], np.float32)
mask1 = cv2.circle(mask1,(image.shape[1]//2,image.shape[0]//2), sz1, 1, -1)

for i in range(3): #채널별로
    fft = np.fft.fft2(noised[...,i])

    # (0,0) frequency: centered
    fft_shift = np.fft.fftshift(fft, axes=[0, 1])
    fft_shift_lpf = fft_shift * mask1
    fft_lpf = np.fft.ifftshift(fft_shift_lpf, axes=[0, 1])
    filtered_lpf = np.fft.ifft2(fft_lpf)
    filtered[...,i] = np.abs(filtered_lpf)

# log10-scale
cv2.imshow('color noised vs filtered', np.hstack([noised,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (6) DCT-Domain (Zonal) Filtering

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised =(image + noise).clip(0,1)
filtered = np.zeros_like(noised)

''' Zonal filter '''
y,x = noised.shape[:2]
mask = np.zeros(noised.shape[:2], dtype=np.float32)
mask = cv2.rectangle(mask, (0, 0), (x//2, y//2), 1, -1)
mask = cv2.circle(mask, (x//2, y//2), x//2, 0, -1)
mask[:,0] = 1
mask[0,:] = 1

for i in range(3):
    dct = cv2.dct(noised[...,i])
    dct *= mask
    filtered[...,i] = cv2.idct(dct)

cv2.imshow('mask', mask)
cv2.imshow('noised vs filtered', np.hstack([noised,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (7) Threshold Coding of DCT Coefficients

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised =(image + noise).clip(0,1)
filtered = np.zeros_like(noised)

for i in range(3):
    dct = cv2.dct(noised[...,i])
    absdct = np.abs(dct)
    ''' Threshold filter '''
    th = np.std(absdct)/2
    mask = np.zeros_like(dct)
    mask[absdct >= th] = 1
    dct *= mask
    filtered[...,i] = cv2.idct(dct)

cv2.imshow('noised vs filtered', np.hstack([noised,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (8) Information in Magnitude vs Angle

In [18]:
img = cv2.imread('../data/Lena.png',0).astype(np.float32)/255

fff = np.fft.fft2(img)
mag, ang = np.abs(fff), np.angle(fff)

only_ang = np.fft.ifft2(np.exp(1j*ang))
only_ang = np.abs(only_ang).astype(np.float32)
only_ang = cv2.normalize(only_ang,None,0,1,cv2.NORM_MINMAX)

only_mag = np.fft.ifft2(mag.astype(np.complex128))
only_mag = np.abs(only_mag).astype(np.float32)
only_mag = cv2.normalize(only_mag,None,0,1,cv2.NORM_MINMAX)

cv2.imshow('mag_only vs phase_only', np.hstack([only_mag,only_ang]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (9) Wavelet Transform

In [44]:
# pip install pywavelets

import cv2
import pywt
import numpy as np
import matplotlib.pyplot as plt

# pywt.families(short=True)  # 제공되는 필터 유형(군)
# pywt.wavelist('bior')      # 해당 유형의 필터군에서 구현되어 있는 필터 세트
wavefilter = 'bior6.8'

image = cv2.imread('../data/Lena.png', 0).astype(np.float32) / 255

LL = image.copy()
for i in range(3):
    LL,_ = pywt.dwt2(LL, wavefilter)
    cv2.imshow('wavelet', cv2.normalize(LL,None,0,1,cv2.NORM_MINMAX))
    cv2.waitKey()
    cv2.destroyAllWindows()

for i in range(3):
    LH = np.zeros_like(LL)
    HL = np.zeros_like(LL)
    HH = np.zeros_like(LL)
    LL = pywt.idwt2((LL, (LH, HL, HH)), wavefilter) 

cv2.imshow('wavelet', np.hstack([image, LL]))
cv2.waitKey()
cv2.destroyAllWindows()